In [3]:
import gensim
import os
import collections
import smart_open
import random
import json
import jieba
from gensim.models import Doc2Vec

In [29]:
def read_data(file_path,tokens_only=False):
    with open(file_path, 'r', encoding='utf-8') as f:

        #words=[]
        lines = f.readlines()
        for line in lines:
            line = json.loads(line)
            question = line['title']
            q_list = jieba.cut(question)
           # words.append(' '.join(q_list).split())
            words = ' '.join(q_list).split()
            for i,line in enumerate(words):
                if tokens_only:
                    yield gensim.utils.simple_preprocess(question)
                else:
                 yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])



file_path = 'E:/litao/Chatbots/data/baike_qa/baike_qa_train.json'
train_corpus = read_data(file_path)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.save('baike_qa2vec.model')




In [30]:
def doc2vec(sent):
    model = Doc2Vec.load('baike_qa2vec.model')
    sent_list = jieba.cut(sent)
    words = ' '.join(sent_list).split()
    sent2vec = model.infer_vector(words) 
    return sent2vec
   

In [4]:
import json
def tem_vec(file_path):
    d2v = []
    an_dict = {}
    f = open(file_path,'r', encoding='utf-8')
    for i,line in enumerate(f.readlines()[:1000]):
        line = json.loads(line)
        question = line['title']
        answer = line['answer']
        v =  doc2vec(question)
       # print(v)
        
        #print (question,answer)
        d2v.append(v)
        an_dict[i] = (question,answer)
    return d2v,an_dict
file_path = 'E:/litao/Chatbots/data/baike_qa/baike_qa_train.json'    


In [5]:
d2v,an_dict = tem_vec(file_path)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.745 seconds.
Prefix dict has been built succesfully.


In [19]:
from scipy.linalg import norm
def vector_similarity(v,d2v,an_dict):
    sims =  []
    sims_dict ={}
    for i,v2 in enumerate(d2v):
                sim = np.dot(v, v2) / (norm(v) * norm(v2))
                sims.append(sim)
                sims_dict[sim] = i
    sims.sort(reverse=True)
    top_n = 3
    for i in range(top_n):
        print (sims[i])
        print(an_dict[sims_dict[sims[i]]])

In [24]:
from time import *
import json

import numpy as np

begin_time = time()  
userinput1 = '松本面板可以配什么品牌的超五类模块??'

userinput3 = '万能的微博，深圳有维修饰品的地方么？'

v1 = doc2vec(userinput1)
v2 = doc2vec(userinput2)
v3 = doc2vec(userinput3)
print('---------') 
vector_similarity(v1,d2v,an_dict)
print('---------') 
vector_similarity(v2,d2v,an_dict)
print('---------') 
vector_similarity(v3,d2v,an_dict)
print('---------') 
end_time=time()
run_time = (end_time-begin_time)/3
print ('Word2vec相似度运行时间：',run_time)

---------
0.9999999348428951
('松本面板可以配什么品牌的超五类模块?? ', 'AMP的试试吧，还有普天的。')
0.4921991317660698
('《项链》的作者是谁 ', '这是莫泊桑的短篇小说. ')
0.41631020173882854
('我家墙面是白色地面是黄色的窗帘买什么颜色的好 ', '橙色')
---------
0.44393269306124394
('成人本科挂名我现在在的学校的信息管理专业是4年脱产成人本科,挂靠 ', '可能是挂靠学校没有本科资格,因此与南农合作办学,所以你应该算是南农成教的学生.你和南农成教的学生可能不在一起上课,但你仍可以拿南农的成人本科证\r\n成人学校也是按分上的,和高考一样\r\n在北京,一些普通高校的成教学院的分就比独立设置的成人高校分数明显高很多.')
0.4425346811938248
('农村家庭用的太阳能热水器哪个牌子好？因为想在农村做太阳能热水器生 ', '现在市面上的品牌很多,大家还是选择皇明的较多.如果经济条件允许还是选择皇明,售后服务会好一些.不知道现在的太阳能热水器是否带泵,因压力不够淋浴时水不足.安装时能配套就更好了.保温管是需要关注的,保证在北方室外温度较低时能正常使用.曾遇到几次早晨没有热水可用的麻烦事. \r\n你可以打他们的服务电话0371-65653835 \r\n')
0.39105994107243985
('如何在比赛中更换阵型？？？ ', '改变阵形在球队管理里面。进入球队管理以后，看左下角。会有一个阵形的小图框。你只需要点上面的“<”或者点“>”就可以改变阵形了。建议你改变阵形以后，设置好替补阵容。然后点下保存。那样你以后都会是那种阵形了\r\n在比赛中暂停，可进行换人、换阵型、设置盯人、罚球，多人模式比赛时需要己方控球才能暂停')
---------
0.45698341715420004
('咳嗽咳嗽还是咳嗽，谁有妙招能治幼儿咳嗽 ', '把一雪梨核去掉，放入四块冰糖，四颗川贝的量（有卖粉的），合上梨。放在铁碗里干蒸，锅里放水。每天都吃一只梨。呵呵，三四天后明显好转 祝你好运')
0.3857136195735292
('我想办一张能透支的银行卡,哪个银行的好,有什么要求,还有哪个银行? ', '办张贷记卡吧,消费透支在一定时期

In [33]:
userinput2 = '万能的微博，深圳有维修饰品的地方么？断了'
userinput1 = '万能的微博，深圳有维修饰品的地方么？断了'
v2 = doc2vec(userinput2)
v1 = doc2vec(userinput1)
print( np.dot(v1, v2) / (norm(v1) * norm(v2)))



1.0000000294002356
